In [6]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from pathlib import Path
import os
import rasterio
from shapely.geometry import MultiPolygon, Polygon
import rioxarray as rio
import xarray as xar

from tqdm import tqdm


os.chdir("/home/weedsci/matt/SemiF-AnnotationPipeline")

from semif_utils.segment_species import Segment
from semif_utils.segment_utils import GenCutoutProps, prep_bbox

from semif_utils.datasets import Cutout, SegmentData

from semif_utils.utils import get_image_meta, apply_mask, crop_cutouts

In [132]:
# Get image jsons
metadir = Path("data/semifield-developed-images/MD_2022-07-20/metadata")
dem_path = Path("/home/weedsci/matt/SemiF-AnnotationPipeline/data/semifield-developed-images/MD_2022-07-20/autosfm/dem/dem_clipped_26986.tif")

metadata = [x for x in metadir.glob("*.json")]
# Get Image Dataclass and preview image array
polys = []
for idx in tqdm(range(len(metadata))):
    imgdata = get_image_meta(metadata[idx])
    bboxes = imgdata.bboxes
    img_polys = []
    for box in bboxes:
        if ("CHAL7" in box.cls["USDA_symbol"]) and (box.is_primary):
            # Get Image Dataclass and preview image array      
            glbox = box.global_coordinates
            # print(glbox)
            poly = Polygon([glbox.top_left, glbox.top_right, glbox.bottom_right, glbox.bottom_left])
            polys.append({"image_id": imgdata.image_id, "geometry": poly})
    # if img_polys:
        # polys.append(img_polys)


100%|██████████| 463/463 [00:03<00:00, 128.63it/s]


In [133]:
print(len(polys))
polys

334


[{'image_id': 'MD_1658327570',
  'geometry': <shapely.geometry.polygon.Polygon at 0x7fe8285bf820>},
 {'image_id': 'MD_1658327570',
  'geometry': <shapely.geometry.polygon.Polygon at 0x7fe7c440d100>},
 {'image_id': 'MD_1658327570',
  'geometry': <shapely.geometry.polygon.Polygon at 0x7fe7c440df10>},
 {'image_id': 'MD_1658327570',
  'geometry': <shapely.geometry.polygon.Polygon at 0x7fe7b1bbbc40>},
 {'image_id': 'MD_1658327570',
  'geometry': <shapely.geometry.polygon.Polygon at 0x7fe7b1bbb550>},
 {'image_id': 'MD_1658327570',
  'geometry': <shapely.geometry.polygon.Polygon at 0x7fe7b1bbb430>},
 {'image_id': 'MD_1658327570',
  'geometry': <shapely.geometry.polygon.Polygon at 0x7fe7b1bbbb20>},
 {'image_id': 'MD_1658327570',
  'geometry': <shapely.geometry.polygon.Polygon at 0x7fe7b1bbb6d0>},
 {'image_id': 'MD_1658327570',
  'geometry': <shapely.geometry.polygon.Polygon at 0x7fe7b1bbbdf0>},
 {'image_id': 'MD_1658328023',
  'geometry': <shapely.geometry.polygon.Polygon at 0x7fe7c4dc1bb0>},


In [29]:
# out_image, transformed = rasterio.mask.mask(raster, polys, nodata=-32767, filled=False,crop=True, invert=False)
from rasterio.plot import show
import geopandas as gpd

raster_xarr = rio.open_rasterio(dem_path, mask_and_scale=True)
clipped = raster_xarr.rio.clip(MultiPolygon(polys).geoms)
clipped.plot()

AttributeError: 'DataArray' object has no attribute 'read'

In [138]:
ids = np.arange(len(polys))
list_polys = [{"geometry": x , "id": y} for x,y in zip(polys, ids)]

gdf = gpd.GeoDataFrame(polys, crs='epsg:9001')
# gdf = gpd.GeoSeries(polys,crs='epsg:26986')

# gdf["ids"] = np.arange(len(polys))
# gdf = gdf.iloc[6]
# gdf.to_file("data/semifield-utils/Poly.shp")
gdf["image_id"]

0      MD_1658327570
1      MD_1658327570
2      MD_1658327570
3      MD_1658327570
4      MD_1658327570
           ...      
329    MD_1658326200
330    MD_1658326200
331    MD_1658326200
332    MD_1658326200
333    MD_1658326200
Name: image_id, Length: 334, dtype: object

In [159]:
from rasterstats import zonal_stats
dem = rasterio.open(dem_path,mask_and_scale=True)
array = dem.read(1)
affine = dem.transform
zs_kallio = zonal_stats(gdf, array, affine=affine, stats=['min', 'max', 'mean', 'median', 'majority', 'range','count'], nodata=-32767)
min = [x["min"] for x in zs_kallio]
max = [x["max"] for x in zs_kallio]
mean = [x["mean"] for x in zs_kallio]
median = [x["median"] for x in zs_kallio]
majority = [x["majority"] for x in zs_kallio]
range = [x["range"] for x in zs_kallio]
count = [x["count"] for x in zs_kallio]

In [163]:
gdf["min"] = min
gdf["max"] = max
gdf["mean"] = mean
gdf["median"] = median
gdf["majority"] = majority
gdf["range"] = range
gdf["count"] = count
gdf = gdf[gdf["count"]!=0]
gdf.to_file("data/semifield-utils/Poly.shp")


In [77]:
listkdf = [{"min": stat["min"], "max": stat["max"], "mean": stat["mean"], "median": stat["median"], "majority": stat["majority"], "id": id, "geometry":geo} for stat,id, geo in zip(zs_kallio, ids,polys )]
listkdf
ngdf = gpd.GeoDataFrame(listkdf, crs='epsg:9001')


In [78]:
ngdf

,min,max,mean,median,majority,id,geometry
0,-0.002812,0.347509,0.078934,0.047032,-0.001785,0,"POLYGON ((0.56721 0.39612, 0.59024 0.39310, 0...."
1,-0.002812,0.347509,0.078934,0.047032,-0.001785,1,"POLYGON ((0.56721 0.39612, 0.59024 0.39310, 0...."
2,-0.002812,0.347509,0.078934,0.047032,-0.001785,2,"POLYGON ((0.56721 0.39612, 0.59024 0.39310, 0...."
3,-0.002812,0.347509,0.078934,0.047032,-0.001785,3,"POLYGON ((0.56721 0.39612, 0.59024 0.39310, 0...."
4,-0.002812,0.347509,0.078934,0.047032,-0.001785,4,"POLYGON ((0.56721 0.39612, 0.59024 0.39310, 0...."
...,...,...,...,...,...,...,...
12235,-0.002812,0.347509,0.078934,0.047032,-0.001785,12235,"POLYGON ((0.56721 0.39612, 0.59024 0.39310, 0...."
12236,-0.002812,0.347509,0.078934,0.047032,-0.001785,12236,"POLYGON ((0.56721 0.39612, 0.59024 0.39310, 0...."
12237,-0.002812,0.347509,0.078934,0.047032,-0.001785,12237,"POLYGON ((0.56721 0.39612, 0.59024 0.39310, 0...."
12238,-0.002812,0.347509,0.078934,0.047032,-0.001785,12238,"POLYGON ((0.56721 0.39612, 0.59024 0.39310, 0...."
